<a href="https://colab.research.google.com/github/tfbf/uW/blob/master/ParseTranslationNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## USFM
[USFM Documentation](https://ubsicap.github.io/usfm/)
## Input file
[Input Repo](https://git.door43.org/Door43-Catalog/hi_tn)
## Expected output
```
\id TIT                                     # Book Id. Must be the first line of the output. Only one **\id** is permitted in the output.
\c 1                                        # Chapter No. Occurs once for each chapter
\p                                          # Para placeholder
\v 1                                        # Verse No
\b                                          # Table separator
\tr                                         # Row Begin
\tc1 Son of God                             # Column: GLQuote
\tc2 Υἱοῦ Θεοῦ                              # Column: OrigQuote
\tc3 guidelines-sonofgodprinciples          # Column: SupportReference
\tr                                         # Row Begin
\tc1-3 यह यीशु के लिए एक महत्वपूर्ण पदवी है।           # Merged-Column: OccurrenceNote
\b                                          # Table separator
\tr                                         # Row Begin
\tc1 that agrees with godliness             # Column: GLQuote
\tc2 τῆς κατ’ εὐσέβειαν                     # Column: OrigQuote
\tc3                                        # Column: SupportReference
\tr                                         # Row Begin
\tc1-3 जो परमेश्वर को आदर देने के लिए उपयुक्त हो       # Merged-Column: OccurrenceNote
\p                                          # Para placeholder

```


# Implemenatation

### Import Python Packages Here

In [0]:
import io
import pandas as pd
import requests

### Program Configurations

In [0]:
col_dtypes = {
    "Book": "category",
    "Chapter": "category",
    "Verse": "category",
    "SupportReference": "object",
    "OrigQuote": "object",
    "GLQuote": "object",
    "OccurrenceNote": "object"
}
columns = list(col_dtypes.keys())
group_by_cols = ["Book", "Chapter", "Verse"]

sep="\t"

src_path = "https://git.door43.org/Door43-Catalog/hi_tn/raw/branch/master/"
src_files = ["hi_tn_42-MRK.tsv", "hi_tn_48-2CO.tsv", "hi_tn_49-GAL.tsv", "hi_tn_57-TIT.tsv", "hi_tn_58-PHM.tsv",
             "hi_tn_61-1PE.tsv", "hi_tn_63-1JN.tsv", "hi_tn_64-2JN.tsv", "hi_tn_65-3JN.tsv", "hi_tn_66-JUD.tsv"]

### Function (_df_to_usfm_) to Convert _pandas_ DataFrame to USFM format

In [0]:
use_newline = False       # Make this flag True to get the output with added newlines as in the Sample given above

def gdf_to_usfm_each(gdf):
  gdf = gdf.sort_values("Index", axis = 0, ascending = True)
  # print(gdf[group_by_cols + ["Index", "GroupOrder"]].head(20))
  df = gdf.reset_index(drop=True)

  # usfm_head = "\\id {0}\n\\c {1}\n\\p\n\\v {2}\n\\b"
  usfm_each = "\\tr\n\\tc1 {0}\n\\tc2 {1}\n\\tc3 {2}\n\\tr\n\\tc1-3 {3}" if use_newline else "\\tr \\tc1 {0} \\tc2 {1} \\tc3 {2}\n\\tr \\tc1-3 {3}"

  verse = None
  chapter = None
  body = []

  for i, r in df.iterrows():
    if verse is None or chapter is None:
      verse = r["Verse"]
      chapter = r["Chapter"]
    body.append(usfm_each.format(r["GLQuote"], r["OrigQuote"], r["SupportReference"], r["OccurrenceNote"]))
  
  return (chapter, "\\v {0}\n\\b\n".format(verse) + "\n\\b\n".join(body))

def df_to_usfm(df, sep_group=False): # sep_group flag when true will separate each Verse group by an extra Newline for better understanding
  # Add an Index column to preserve order of individual records
  df["Index"] = df.index
  # Group Order column to preserver group order
  df["GroupOrder"] = pd.factorize(df["Chapter"].str.cat(df["Verse"], sep =":|:"))[0]
  
  group_df = df.groupby(["Book", "GroupOrder"])
  # group_df = df.query('Chapter=="1" and Verse=="1"').groupby(["Book", "GroupOrder"])   # Just testing with Chapter 1 and Verse 1

  bk = None
  ch = None
  output = ""
  vs = []

  for i, each in group_df.apply(gdf_to_usfm_each).iteritems():
    if bk is None or bk != i[0]:
      bk = i[0]
      output += ("\\id {0}".format(bk))
    if ch is None or ch != each[0]:
      ch = each[0]
      output += ("\n\\p\n".join(vs))
      output += ("\n\\c {0}\n".format(ch))
      vs = []
    vs.append(each[1])
    # print(i[0], i[1])
  print(output)

### Iterate through each file and apply the function

In [0]:
for src_file in src_files[3:4]: # Let's test on Titus first. Once the code functions, you can remove the slicing operation, so that it will convert the entire set of files.
    # Fetch data from Url
    s = requests.get(src_path + src_file).content
    # Load pandas DataFrame
    tnotes = pd.read_csv(io.StringIO(s.decode('utf-8')), delimiter=sep)
    
    # Filter with the given columns
    tnotes = tnotes[columns]
    # Fill NaN with empty string
    tnotes = tnotes.fillna("")
    # Enforce the given Col datatypes for better performance and data stability
    tnotes = tnotes.astype(col_dtypes)

    # # Convert tnotes dataframe to usfm data
    # tnotes_usfm = df_to_usfm(tnotes, sep_group=True)
    # # Save the usfm data to the following file path
    # save_file = "{0}.usfm".format(src_file.split('.')[0])
    # with open(save_file, "w") as f:
    #   f.write(tnotes_usfm)

# Solution Sample Output

### Display sample records in DataFrame

In [0]:
tnotes.head(20)

Book  ...                                     OccurrenceNote
0   TIT  ...  # तीतुस का परिचय<br>## भाग 1: सामान्य परिचय<br...
1   TIT  ...  # (rc://hi/ta/man/translate/figs-abstractnouns...
2   TIT  ...                        विश्वास को मजबूत करने क लिए
3   TIT  ...          जो परमेश्वर को आदर देने के लिए उपयुक्त हो
4   TIT  ...                            समय के प्रारम्भ से पहले
5   TIT  ...                                        उचित समय पर
6   TIT  ...  पौलूस परमेश्वर के सन्देश की इस तरह बात करता है...
7   TIT  ...  उसने मुझ पर भरोसा किया कि मैं आगे ले जाऊं या “...
8   TIT  ...                  परमेश्वर, जो हमारा उद्धार करता है
9   TIT  ...  यद्यपि तीतुस पौलूस का जैविक पुत्र नहीं था, तो ...
10  TIT  ...  पौलूस मसीह में उसी विश्वास को प्रगट करता है जि...
11  TIT  ...  यह एक सामान्य अभिवादन था जिसका पौलूस उपयोग किय...
12  TIT  ...                  मसीह यीशु जो हमारा उद्धारकर्ता है
13  TIT  ...                                         इस कारण से
14  TIT  ...         मैंने तुझे क्रेते में रुके रहने के लिए कहा
15  TIT  ...  ताकि तू उन बातों को सही करने का काम पूरा करे ज...
16  TIT  ...  प्राचीनों को नियुक्त करना या “प्राचीन निर्वाचि...
17  TIT  ...  प्रारम्भिक मसीही कलीसियाओं में, मसीही प्राचीन ...
18  TIT  ...  तीतुस को क्रेते द्वीप के प्रत्येक नगर में प्रा...
19  TIT  ...  “निर्दोष” का मतलब एक ऐसा व्यक्ति जो बुरे काम न...

[20 rows x 7 columns]

### DataFrame column types

In [0]:
tnotes.dtypes

Book                category
Chapter             category
Verse               category
SupportReference      object
OrigQuote             object
GLQuote               object
OccurrenceNote        object
dtype: object

### Grouped by Book, Chapter and Verse

In [0]:
tnotes.groupby(group_by_cols).describe().head(20)

SupportReference  ... OccurrenceNote
                              count  ...           freq
Book Chapter Verse                   ...               
TIT  1       1                    2  ...              1
             10                   4  ...              1
             11                   4  ...              1
             12                   3  ...              1
             13                   2  ...              1
             14                   2  ...              1
             15                   3  ...              1
             16                   2  ...              1
             2                    1  ...              1
             3                    4  ...              1
             4                    4  ...              1
             5                    5  ...              1
             6                    4  ...              1
             7                    4  ...              1
             8                    2  ...              1
             9                    2  ...              1
             intro                1  ...              1
     2       1                    3  ...              1
             10                   4  ...              1
             11                   2  ...              1

[20 rows x 16 columns]

### _df_to_usfm_ Output

In [0]:
df_to_usfm(tnotes, sep_group=True)    # This function can be used in the loop above where we read the data from input tsv files

\id TIT
\c front
\v intro
\b
\tr \tc1  \tc2  \tc3 
\tr \tc1-3 # तीतुस का परिचय<br>## भाग 1: सामान्य परिचय<br><br>### तीतुस की पुस्तक की रूपरेखा<br><br>1। पौलूस तीतुस को धर्मी अगुवे नियुक्त करने के निर्देश देता है (1:1-16)<br>1। पौलूस तीतुस को निर्देश देता है कि वह लोगों को धर्मी जीवन जीने के लिए प्रशिक्षित करे (2:1-3:11)<br>1। पौलूस अंत में अपनी कुछ योजनाएं बताता है और विभिन्न विश्वासियों को शुभकामनाएं भेजता है (3:12-15)<br><br>#### तीतुस की पुस्तक किसने लिखी?<br><br>पौलूस ने तीतुस की पुस्तक को लिखा। पौलूस तरसुस नगर का निवासी था। वह अपने प्रारंभिक जीवन में शाऊल के नाम से जाना जाता था। मसीही बनने से पहले पौलूस एक फरीसी था। उसने मसीहियों को सताया। मसीही बनने के बाद, उसने पूरे रोमी साम्राज्य में लोगों को यीशु के बारे में बताते हुए अनेक यात्राएँ कीं.<br><br>### तीतुस की पुस्तक किस बारे में है?<br><br>पौलूस ने यह पत्र अपने सहकर्मी, तीतुस को लिखा जो क्रेते द्वीप पर कलीसियाओं का नेतृत्व करता था। पौलूस उसे कलीसियाओं में अगुवे नियुक्त करने के निर्देश देता है। पौलूस इस बात का भी वर्णन करता है कि